In [ ]:
!pip install mistralai tqdm psycopg2-binary sqlalchemy python-dotenv pandas termcolor openpyxl black --quiet

%load_ext autoreload
%autoreload 2

from database.matching import find_matches, compare_names
from database.getdata import query_data, get_connection, execute_query
import pandas as pd
import os
from mistralai import Mistral
from dotenv import load_dotenv
import tqdm

import json

load_dotenv()

# CHARGEMENT DES DONNEES FILMS, REALISATEURS, PRODUCTEURS, DISTRIBUTEURS

## LOAD DATASET Dataset5050_CNC Films Agréés 2003-2024.xlsx

In [ ]:
# use openpyxl to get list of spreadsheet names
filename = "../database/data/Dataset5050_CNC Films Agréés 2003-2024.xlsx"
xls = pd.ExcelFile(filename)
sheet_names = xls.sheet_names
print(f"{len(sheet_names)} feuilles trouvés dans le fichier {filename}")
sheet_names = [sheet for sheet in sheet_names if "20" in sheet]
sheet_names

In [ ]:
df = pd.read_excel(filename, sheet_name=sheet_names[0], skiprows=4)
df.head()

### REALISATEURS

In [ ]:
all_realisateurs = []

for sheet in tqdm.tqdm(sheet_names, desc="Processing sheets"):
    df = pd.read_excel(filename, sheet_name=sheet, skiprows=4)

    realisateurs = [
        [name.strip() for name in tt.split("/")]
        for tt in df["REALISATEUR"].fillna("").tolist()
    ]
    realisateurs = [item for sublist in realisateurs for item in sublist]

    all_realisateurs.extend(realisateurs)

all_realisateurs = list(set(all_realisateurs))
all_realisateurs.sort()
all_realisateurs.remove("")
print(f"{len(all_realisateurs)} réalisateurs trouvés dans le fichier {filename}")

In [ ]:
df_realisateurs = query_data(
    "SELECT * FROM inegalites_cinema.cheffes_postes WHERE type_de_poste = 'Réalisation'"
)

In [ ]:
find_matches("Ékoue", df_realisateurs, "nom")

In [ ]:
realisateur_to_check = []
realisateurs_matched = []  # ['GABARSKI Sam', 'GARBARSKI Sam', 'GAUD Jean-Paul']

for ii, realisateur in tqdm.tqdm(
    enumerate(all_realisateurs),
    desc="Processing realisateurs",
    total=len(all_realisateurs),
):
    matches = find_matches(realisateur, df_realisateurs[ii:], "nom", threshold=0.90)
    if len(matches) > 1:
        print(f"Realisateur: {realisateur}")
        print(matches)
        print()
        realisateur_to_check.append(realisateur)
    elif len(matches) == 1:
        realisateurs_matched.append(realisateur)
    elif len(matches) == 0:
        df_toadd = pd.DataFrame([{"nom": realisateur, "type_de_poste": "Réalisation"}])
        df_toadd.to_sql(
            "cheffes_postes",
            con=get_connection(),
            schema="inegalites_cinema",
            if_exists="append",
            index=False,
        )
        df_realisateurs = pd.concat([df_realisateurs, df_toadd], ignore_index=True)

In [ ]:
df_realisateurs = query_data(
    "SELECT id, nom FROM inegalites_cinema.cheffes_postes where type_de_poste='Réalisation'"
)

In [ ]:
# find duplicated entries

ids_to_delete = []
for realisateur in tqdm.tqdm(
    df_realisateurs["nom"].to_list(), desc="Processing realisateurs"
):
    matches = find_matches(realisateur, df_realisateurs, "nom", threshold=0.9)
    if len(matches) > 1:
        ids = matches["id"].tolist()
        ids_to_delete.extend(ids[1:])

    break
ids_to_delete = list(set(ids_to_delete))

In [ ]:
with open("../database/realisateurs_matched.txt", "w") as f:
    for item in realisateurs_matched:
        f.write("%s\n" % item)

In [ ]:
realisateur_to_check

In [ ]:
len(ids_to_delete)
for ii in tqdm.tqdm(ids_to_delete):
    execute_query(f"DELETE FROM inegalites_cinema.cheffes_postes WHERE id = {ii}")

In [ ]:
df_realisateurs = query_data(
    "SELECT id, type_de_poste, nom FROM inegalites_cinema.cheffes_postes where type_de_poste='Réalisation'"
)
df_realisateurs.head()

In [ ]:
find_matches("José ALCALA", df_realisateurs, "nom")

### PROCESS FILMS DATASET

In [ ]:
df_film = pd.read_excel(filename, sheet_name=sheet_names[0], skiprows=4)
df_film.columns = [col.lower().replace(" ", "_") for col in df_film.columns]
df_film["realisateur"] = [
    [name.strip() for name in tt.split("/")]
    for tt in df_film["realisateur"].fillna("").tolist()
]
df_film = df_film.explode("realisateur")
df_film.columns = [
    "n_visa_exploitation",
    "nom_originel",
    "realisateur",
    "producteurs",
    "devis",
    "genre",
    "eof",
    "rang",
    "bonus_parite_",
    "asr",
    "payant",
    "clair",
    "sofica",
    "credit_dimpot",
    "aide_regionale",
    "nationalite",
]
df_film.head()

In [ ]:
df_realisateurs = query_data(
    "select id, nom FROM inegalites_cinema.cheffes_postes where type_de_poste='Réalisation'"
)
df_realisateurs.head(3)

In [ ]:
columns_clean = df_film.columns.tolist()
columns_clean.remove("realisateur")

df_film_clean = pd.DataFrame(
    [],
    columns=[
        "n_visa_exploitation",
        "nom_originel",
        "realisateur",
        "producteurs",
        "devis",
        "genre",
        "id_realisateur",
    ],
)

In [ ]:
realisateur_to_check = []
for sheet in sheet_names:
    df_film = pd.read_excel(filename, sheet_name=sheet, skiprows=4)
    df_film.columns = [col.lower().replace(" ", "_") for col in df_film.columns]
    df_film["realisateur"] = [
        [name.strip() for name in tt.split("/")]
        for tt in df_film["realisateur"].fillna("").tolist()
    ]
    df_film = df_film.explode("realisateur")
    df_film.columns = [
        "n_visa_exploitation",
        "nom_originel",
        "realisateur",
        "producteurs",
        "devis",
        "genre",
        "eof",
        "rang",
        "bonus_parite_",
        "asr",
        "payant",
        "clair",
        "sofica",
        "credit_dimpot",
        "aide_regionale",
        "nationalite",
    ]
    df_film = df_film[
        [
            "n_visa_exploitation",
            "nom_originel",
            "realisateur",
            "producteurs",
            "devis",
            "genre",
        ]
    ]

    # iterate over df_film, match the realisateur with the database and add the id to the dataframe
    for index, row in tqdm.tqdm(
        df_film.iterrows(), total=len(df_film), desc=f"Processing {sheet}"
    ):
        realisateur = row["realisateur"]
        matches = find_matches(realisateur, df_realisateurs, "nom", threshold=0.9)
        if len(matches) == 1:
            df_toadd = pd.DataFrame([row])
            df_toadd["id_realisateur"] = matches.iloc[0]["id"]
            df_film_clean = pd.concat([df_film_clean, df_toadd], ignore_index=True)
        else:
            print(f"Realisateur: {realisateur}")
            print(matches)
            print()
            realisateur_to_check.append(realisateur)

    break